In [1]:
import os
import pandas as pd

In [2]:
def obtener_ultima_linea(csv_path):
    with open(csv_path, 'r') as archivo:
        lineas = archivo.readlines()
        if lineas:
            ultima_linea = lineas[-1].strip()
            return ultima_linea
        else:
            return None

In [3]:
def procesar_directorio(directorio):
    datos = []

    for archivo in os.listdir(directorio):
        if archivo.endswith(".csv"):
            ruta_completa = os.path.join(directorio, archivo)
            ultima_linea = obtener_ultima_linea(ruta_completa)

            if ultima_linea:
                datos_ultima_linea = ultima_linea.split(',')

                fecha_hora = pd.to_datetime(datos_ultima_linea[0])
                latitud = float(datos_ultima_linea[1])
                longitud = float(datos_ultima_linea[2])

                partes_nombre = archivo[:-4].split('_')
                try:
                    ascend = float(partes_nombre[0])
                    descend = float(partes_nombre[1])
                    altitude = float(partes_nombre[2])
                except ValueError:
                    print(f"Advertencia: No se pudieron extraer valores de ascend, descend y altitude para {archivo}")
                    continue

                datos.append({
                    'Fecha_Hora': fecha_hora,
                    'Latitud': latitud,
                    'Longitud': longitud,
                    'Ascend': ascend,
                    'Descend': descend,
                    'MaxAltitude': altitude
                })

    df = pd.DataFrame(datos)
    return df

In [4]:
directorio_csv = "tocos"

df_resultado = procesar_directorio(directorio_csv)

display(df_resultado)

,Fecha_Hora,Latitud,Longitud,Ascend,Descend,MaxAltitude
0,2023-12-27 12:51:11.250000+00:00,-24.91973,-55.84949,2.0,3.0,20000.0
1,2023-12-27 13:01:01.875000+00:00,-24.91022,-55.87806,2.0,3.0,21000.0
2,2023-12-27 13:10:37.500000+00:00,-24.90642,-55.92925,2.0,3.0,22000.0
3,2023-12-27 13:19:57.187500+00:00,-24.91121,-55.99980,2.0,3.0,23000.0
4,2023-12-27 13:29:25.312500+00:00,-24.92307,-56.08823,2.0,3.0,24000.0
...,...,...,...,...,...,...
127,2023-12-27 11:24:50.625000+00:00,-25.21631,-56.75912,4.0,6.0,26000.0
128,2023-12-27 11:29:26.250000+00:00,-25.22363,-56.80552,4.0,6.0,27000.0
129,2023-12-27 11:33:53.437500+00:00,-25.22979,-56.84810,4.0,6.0,28000.0
130,2023-12-27 11:38:26.250000+00:00,-25.23293,-56.88728,4.0,6.0,29000.0


In [5]:
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point
import geopandas as gpd

def get_map(df_resultado, output_path):
    # Crear un GeoDataFrame a partir del DataFrame df_resultado
    latitudes = list(df_resultado['Latitud'])
    longitudes = list(df_resultado['Longitud'])
    geometry = [Point(xy) for xy in zip(longitudes, latitudes)]
    geo_df = gpd.GeoDataFrame(df_resultado, geometry=geometry)
    m = folium.Map(location=[geo_df['Latitud'].mean(), geo_df['Longitud'].mean()], zoom_start=10)

    # Convertir GeoDataFrame a una lista de (latitud, longitud, peso) para HeatMap
    heat_data = [(row['Latitud'], row['Longitud']) for index, row in geo_df.iterrows()]

    HeatMap(heat_data, radius=15).add_to(m)
    os.makedirs(output_path, exist_ok=True)
    m.save(os.path.join(output_path, 'heatmap.html'))


get_map(df_resultado, 'projects/heatmap/')